In [ ]:
import os 
import re

import pandas as pd
import numpy as np
import scanpy as sc


from SURE import DensityFlow
from SURE.perturb import DoseMatrix,LabelMatrix

import torch
torch.set_float32_matmul_precision("high")

import seaborn as sns
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 仅显示GPU 0

TF_pairs = ['Pparg-Runx2','Runx2-Pparg',
            'Cebpa-Pparg','Pparg-Cebpa',
            'Cebpa-Mycn','Mycn-Cebpa',
            'Cebpa-Myog','Myog-Cebpa',
            'Mycn-Myog','Myog-Mycn',
            'Mycn-Runx2','Runx2-Mycn',
            'Mycn-Pparg','Pparg-Mycn']

In [ ]:
adata_ = sc.read_h5ad('scTF_seq_adata.h5ad')
adata_ = adata_.raw.to_adata()
adata = adata_.copy()

def count_pert(row):
    if row['TF'] in TF_pairs:
        return 2
    elif row['TF']=='D0':
        return 0
    else:
        return 1
    
def norm_pert(row):
    if row['TF'] in TF_pairs:
        return row['TF'].replace('-','+')
    else:
        return row['TF']

adata.obs['nperts'] = adata.obs.apply(count_pert, axis=1)
adata.obs['target'] = adata.obs.apply(norm_pert, axis=1)



In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.shape

(45978, 55536)

In [ ]:
pert_col = 'target'

xs = adata.X

lb1 = DoseMatrix()
labels = adata.obs[pert_col].tolist()
label_dose = adata.obs['Dose'].tolist()
us1 = lb1.fit_transform(labels, label_dose, sep_pattern='@')
ln1 = lb1.labels_


lb2 = LabelMatrix()
us2 = lb2.fit_transform(adata.obs['Phase_corrected'], 'G1')
ln2 = lb2.labels_

lb3 = LabelMatrix()
us3 = lb3.fit_transform(adata.obs['batch'],'exp12-13')
ln3 = lb3.labels_

us = np.hstack([us1,us2,us3])
ln = np.hstack([ln1,ln2,ln3])
us.shape


(45978, 404)

In [ ]:
model = DensityFlow(input_size = xs.shape[1],
                      cell_factor_size=us.shape[1],
                      use_cuda=True)

🧬 DensityFlow Initialized:
   - Codebook size: 15
   - Latent Dimension: 50
   - Gene Dimension: 55536
   - Hidden Dimensions: [512]
   - Device: cuda:0
   - Parameters: 78,632,111


In [ ]:
%%time 

model.fit(xs, 
          us=us, 
          use_jax=True)

Training: 100%|██████████| 100/100 [1:03:08<00:00, 37.89s/epoch, loss=7237587.7935]

CPU times: user 1h 2min 15s, sys: 11min 13s, total: 1h 13min 28s
Wall time: 1h 3min 17s


In [7]:
DensityFlow.save_model(model, 'dose_densityflow_model2.pth')

Model saved to /home/zfeng/workspace/zfeng/Workspace/PerturbFlow_Ex/scTF_seq/dose_densityflow_model2.pth
